In [1]:
import input_data
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

C:\Users\Joee\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [2]:
import tensorflow as tf
sess = tf.InteractiveSession()

In [3]:
x = tf.placeholder("float", [None, 784])
y = tf.placeholder("float", [None, 10])

**基础模型**

In [6]:
W = tf.Variable(tf.zeros([784, 10]))
b = tf.Variable(tf.zeros([10]))

In [7]:
sess.run(tf.initialize_all_variables())

Instructions for updating:
Use `tf.global_variables_initializer` instead.


In [8]:
y_ = tf.nn.softmax(tf.matmul(x, W) + b)

In [9]:
cross_entropy = -tf.reduce_sum(y*tf.log(y_))

In [10]:
train_step = tf.train.GradientDescentOptimizer(0.01).minimize(cross_entropy)

In [13]:
for i in range(1000):
    batch = mnist.train.next_batch(50)
    train_step.run(feed_dict={x: batch[0], y: batch[1]})

In [14]:
correct = tf.equal(tf.argmax(y, 1), tf.argmax(y_, 1))
accuracy = tf.reduce_mean(tf.cast(correct, "float"))

In [15]:
print(accuracy.eval(feed_dict={x: mnist.test.images, y: mnist.test.labels}))

0.9206


**多层卷积模型**

In [16]:
# 变量初始化函数
def weight_variable(shape):
    initial = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(initial)

def bias_variable(shape):
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial)

In [17]:
# 卷积和池化函数
def conv_2d(x, W):
    return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')

def max_pool_2x2(x):
    return tf.nn.max_pool(x, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')

In [18]:
x_image = tf.reshape(x, [-1, 28, 28, 1])

In [19]:
W1 = weight_variable([5, 5, 1, 32])
b1 = bias_variable([32])

In [20]:
hc1 = tf.nn.relu(conv_2d(x_image, W1) + b1)
hp1 = max_pool_2x2(hc1)

In [21]:
W2 = weight_variable([5, 5, 32, 64])
b2 = bias_variable([64])

In [22]:
hc2 = tf.nn.relu(conv_2d(hp1, W2) + b2)
hp2 = max_pool_2x2(hc2)

In [23]:
Wfc1 = weight_variable([7*7*64, 1024])
bfc1 = bias_variable([1024])

In [25]:
hp2_flat = tf.reshape(hp2, [-1, 7*7*64])
hfc1 = tf.nn.relu(tf.matmul(hp2_flat, Wfc1) + bfc1)

In [26]:
keep_prob = tf.placeholder("float")
hfc1_drop = tf.nn.dropout(hfc1, keep_prob)

In [27]:
Wfc2 = weight_variable([1024, 10])
bfc2 = bias_variable([10])

In [28]:
y_ = tf.nn.softmax(tf.matmul(hfc1_drop, Wfc2) + bfc2)

In [29]:
cross_entropy = -tf.reduce_sum(y*tf.log(y_))

In [30]:
train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)

In [31]:
correct = tf.equal(tf.argmax(y, 1), tf.argmax(y_, 1))
accuracy = tf.reduce_mean(tf.cast(correct, "float"))

In [33]:
sess.run(tf.initialize_all_variables())

In [40]:
for i in range(20000):
    batch = mnist.train.next_batch(50)
    test_batch = mnist.test.next_batch(50)
    train_step.run(feed_dict={x: batch[0], y: batch[1], keep_prob: 0.5})
    if i % 100 == 0:
        log = 'step {}: train acc is {}'.format(i, accuracy.eval(feed_dict={x: batch[0], y: batch[1], keep_prob: 1.0}))
        print(log)

step 0: train acc is 0.1599999964237213
step 100: train acc is 0.9599999785423279
step 200: train acc is 0.800000011920929
step 300: train acc is 0.8999999761581421
step 400: train acc is 0.9599999785423279
step 500: train acc is 0.8999999761581421
step 600: train acc is 1.0
step 700: train acc is 0.9800000190734863
step 800: train acc is 0.9800000190734863
step 900: train acc is 0.9800000190734863
step 1000: train acc is 0.9200000166893005
step 1100: train acc is 0.9399999976158142
step 1200: train acc is 0.9599999785423279
step 1300: train acc is 0.9599999785423279
step 1400: train acc is 0.9800000190734863
step 1500: train acc is 0.9599999785423279
step 1600: train acc is 0.9599999785423279
step 1700: train acc is 1.0
step 1800: train acc is 1.0
step 1900: train acc is 0.9599999785423279
step 2000: train acc is 0.9800000190734863
step 2100: train acc is 1.0
step 2200: train acc is 0.9800000190734863
step 2300: train acc is 1.0
step 2400: train acc is 0.9599999785423279
step 2500: tr

In [42]:
print('total test acc: {}.'.format(accuracy.eval(feed_dict={x: mnist.test.images, y: mnist.test.labels, keep_prob: 1.0})))

total test acc: 0.9925000071525574.
